In [1]:
# Importing necessary libraries
from osgeo import gdal
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster
from skimage import io, color

In [2]:
#Creating variable for working directory
path_var="C:\\Summer2022\\Marginal-Land-Classification\\palakkad_6bands.tif"

In [3]:
#Reading the data
original_data=gdal.Open(path_var)

In [4]:
#Converting raster data to numpy array
def read_convert():
    img_h=original_data.RasterYSize
    img_w=original_data.RasterXSize
    band_number=original_data.RasterCount
    data = np.empty(((img_w)*(img_h), band_number))
    for i in range(0,band_number):
        get_band=original_data.GetRasterBand(i+1)
        get_array=get_band.ReadAsArray()
        data[:, i-1] = get_array.flatten()
    return data,img_h,img_w
#         im_arr.append(get_array)
#     return im_arr,img_h,img_w,band_number
np_arr,hy,wx=read_convert()    

In [5]:
#KMeans algorithm creation and fitting.
n_c=7
km=cluster.KMeans(n_c)
km.fit(np_arr)

KMeans(n_clusters=7)

In [6]:
#Extracting labels after fitting of kmeans
res = km.labels_.reshape((hy,wx))

In [7]:
#Mapping cluster results to the original area in a new tif file
drv=gdal.GetDriverByName("GTiff")
drv.Register()
clf_pal=drv.Create("C:\\Summer2022\\Marginal-Land-Classification\\palakkad_6bands_classified_new.tif",xsize=wx,ysize=hy,bands=1,eType=gdal.GDT_Float32)
clf_pal.SetGeoTransform(original_data.GetGeoTransform())
clf_pal.SetProjection(original_data.GetProjection())
clf_pal.GetRasterBand(1).WriteArray(res)
clf_pal.GetRasterBand(1).SetNoDataValue(-9999.0)
clf_pal.FlushCache()
clf_pal=None